In [51]:
from feijoa import create_job, Experiment, SearchSpace, Real
import pandas as pd

space = SearchSpace()
space.insert(Real("x", low=0.0, high=5.0))
space.insert(Real("y", low=0.0, high=2.0))


def bill(x, y):
    return (
        (1.5 - x + x * y) ** 2
        + (2.25 - x + x * y**2) ** 2
        + (2.625 - x + x * y**3) ** 2
    )


def objective(experiment: Experiment):
    x = experiment.params.get("x")
    y = experiment.params.get("y")

    return bill(x, y)

In [37]:
job = create_job(search_space=space)

In [38]:
job.do(objective, n_trials=1500)

/Users/qnbhd/Development/feijoa/feijoa/search/algorithms/skopt.py:60: UserWarning: Now per-emit count in skopt technique implemented not correctly. Per-emit count is property, return default value (1).
  warnings.warn('Now per-emit count in skopt technique implemented not correctly.'


In [39]:
df = job.dataframe

In [40]:
job.best_parameters

{'x': 2.9932797233033757, 'y': 0.4980567102896055}

In [42]:
import plotly.graph_objects as go
import pandas as pd

In [72]:
import numpy as np

x = np.linspace(-2.0, 5.0, 70)
y = np.linspace(-2.0, 5.0, 70)

X, Y = np.meshgrid(x, y)

Z = bill(X, Y)

layout = go.Layout(
    width=700, height=700, title_text="Chasing global Minima"
)
fig = go.Figure(
    data=[go.Surface(x=X, y=Y, z=Z, colorscale="Blues")],
    layout=layout,
)

fig.update_traces(
    contours_z=dict(
        show=True,
        usecolormap=True,
        highlightcolor="limegreen",
        project_z=True,
    )
)

fig.add_scatter3d(
    x=df["x"],
    y=df["y"],
    z=df["objective_result"],
    mode="markers",
    marker=dict(
        size=5, color=-df["objective_result"], colorscale="Reds"
    ),
)

In [68]:
import pandas as pd

ddf = pd.DataFrame(columns=["finish_time", "objective_result"])

print(ddf)

minima = 1e100

for index, rows in df.iterrows():
    if rows["objective_result"] <= minima:
        minima = rows["objective_result"]
        ddf = ddf.append(
            {
                "finish_time": rows["finish_time"],
                "objective_result": rows["objective_result"],
            },
            ignore_index=True,
        )

Empty DataFrame
Columns: [finish_time, objective_result]
Index: []


/var/folders/gp/1qc74rb50b9cp66q8cxshnwh0000gn/T/ipykernel_40273/1987797541.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/gp/1qc74rb50b9cp66q8cxshnwh0000gn/T/ipykernel_40273/1987797541.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/gp/1qc74rb50b9cp66q8cxshnwh0000gn/T/ipykernel_40273/1987797541.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/gp/1qc74rb50b9cp66q8cxshnwh0000gn/T/ipykernel_40273/1987797541.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/gp/1qc74rb50b9cp66q8cxshnwh0000gn/T/ipykernel_40273/1987797541.py:12: FutureWarning:

The frame.append method is de

In [69]:
ddf

,finish_time,objective_result
0,2022-04-21 18:16:54.762275,25.230988
1,2022-04-21 18:16:54.788561,6.420967
2,2022-04-21 18:16:54.840507,6.177193
3,2022-04-21 18:16:54.857094,2.029734
4,2022-04-21 18:16:54.904833,0.842451
5,2022-04-21 18:16:56.808002,0.741779
6,2022-04-21 18:16:57.272778,0.637984
7,2022-04-21 18:16:57.989100,0.586827
8,2022-04-21 18:16:58.416692,0.537879
9,2022-04-21 18:16:58.566593,0.03532


In [71]:
layout = dict(
    title="Bill's function optimization",
    xaxis=dict(title="Autotuning time (date)"),
    yaxis=dict(title="Execution time (seconds)"),
)

fig = go.Figure(layout=layout)

fig.add_scatter(
    x=ddf["finish_time"],
    y=ddf["objective_result"],
    name="Finded minima",
    mode="markers",
)